In [2]:
# Load necessary library and module
import warnings
warnings.filterwarnings('ignore')

import sys
import pandas as pd
import seaborn as sns
import numpy as np
import re
import string
import unicodedata
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from textblob import TextBlob, Word

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# Apply text pre-processing to DataFrame
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
snowball = nltk.stem.SnowballStemmer('english')
porter_stemmer = PorterStemmer()

from textblob import TextBlob, Word

In [4]:
# import StemmerFactory class
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
# stemming process
sentence = 'Perekonomian Indonesia sedang dalam pertumbuhan yang membanggakan'
output   = stemmer.stem(sentence)

In [5]:
# stemming(sentence)
print(output)

ekonomi indonesia sedang dalam tumbuh yang bangga


## LOAD DATASET

In [18]:
dataset= pd.read_csv('dataset/dataset_after_nomorurut_23112023_alldataset.csv', encoding='ISO-8859-1')#'UTF-8') #'ISO-8859-1')
# samples= pd.read_csv('hasil/samples.csv', encoding='ISO-8859-1')

In [19]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2324 entries, 0 to 2323
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2324 non-null   float64
 1   likes              2324 non-null   int64  
 2   quotes             2324 non-null   int64  
 3   replies            2324 non-null   int64  
 4   retweets           2324 non-null   int64  
 5   searchQuery        2324 non-null   object 
 6   text               2324 non-null   object 
 7   timestamp          2324 non-null   object 
 8   tanggal            2324 non-null   object 
 9   url                2324 non-null   object 
 10  user_description   1726 non-null   object 
 11  user_join          2324 non-null   object 
 12  user_location      1223 non-null   object 
 13  user_totalfollwer  2324 non-null   int64  
 14  user_following     2324 non-null   int64  
 15  user_totalikes     2324 non-null   int64  
 16  user_tweets        2324 

In [12]:
from datetime import datetime

In [16]:
# datetime_object = datetime.strptime(dataset['tanggal'], '%Y/%m/%d')

In [20]:
# bank = bank[['content', 'score']]
# samples=dataset[['tanggal','text']].iloc[1:10]
samples=dataset[['tanggal','text','likes','replies','quotes','retweets','searchQuery','timestamp','user_location']]

In [21]:
samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2324 entries, 0 to 2323
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tanggal        2324 non-null   object
 1   text           2324 non-null   object
 2   likes          2324 non-null   int64 
 3   replies        2324 non-null   int64 
 4   quotes         2324 non-null   int64 
 5   retweets       2324 non-null   int64 
 6   searchQuery    2324 non-null   object
 7   timestamp      2324 non-null   object
 8   user_location  1223 non-null   object
dtypes: int64(4), object(5)
memory usage: 163.5+ KB


In [22]:
#lowercase all text
samples['text_lower']=(dataset['text'].str.lower())

In [24]:
print(samples.iloc[10])

tanggal                                                 11/22/2023
text             Buat yang bertanya-tanya, kenapa sih Prabowo-G...
likes                                                           13
replies                                                          4
quotes                                                           1
retweets                                                         9
searchQuery                                               #prabowo
timestamp                                 2023-11-22T04:38:00.000Z
user_location                               DKI Jakarta, Indonesia
text_lower       buat yang bertanya-tanya, kenapa sih prabowo-g...
Name: 10, dtype: object


In [25]:
samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2324 entries, 0 to 2323
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tanggal        2324 non-null   object
 1   text           2324 non-null   object
 2   likes          2324 non-null   int64 
 3   replies        2324 non-null   int64 
 4   quotes         2324 non-null   int64 
 5   retweets       2324 non-null   int64 
 6   searchQuery    2324 non-null   object
 7   timestamp      2324 non-null   object
 8   user_location  1223 non-null   object
 9   text_lower     2324 non-null   object
dtypes: int64(4), object(6)
memory usage: 181.7+ KB


In [ ]:
# print(stemming(samples.text[1]))

In [26]:
samples.head()

,tanggal,text,likes,replies,quotes,retweets,searchQuery,timestamp,user_location,text_lower
0,11/21/2023,Analisis yang keren banget dari Bang Hasan Nas...,135,32,5,84,#indonesiamaju,2023-11-21T17:31:00.000Z,"DKI Jakarta, Indonesia",analisis yang keren banget dari bang hasan nas...
1,11/21/2023,"Mari kita bersatu, kita fokus melihat ke depan...",256,69,5,103,#indonesiamaju,2023-11-21T09:06:00.000Z,Indonesia Raya,"mari kita bersatu, kita fokus melihat ke depan..."
2,11/22/2023,Gubernur BI : Pertumbuhan Ekonomi Indonesia Te...,1,0,0,0,#indonesiamaju,2023-11-22T06:40:00.000Z,"Bangka Belitung, Indonesia",gubernur bi : pertumbuhan ekonomi indonesia te...
3,11/22/2023,Gubernur BI : Pertumbuhan Ekonomi Indonesia Te...,1,0,0,0,#indonesiamaju,2023-11-22T06:39:00.000Z,"Bangka Belitung, Indonesia",gubernur bi : pertumbuhan ekonomi indonesia te...
4,11/22/2023,Stabilitas Pertumbuhan Ekonomi Indonesia \n\n#...,1,0,0,0,#indonesiamaju,2023-11-22T06:39:00.000Z,"Bangka Belitung, Indonesia",stabilitas pertumbuhan ekonomi indonesia \n\n#...


## STOPWORD REMOVAL

In [11]:
# samplex= pd.read_csv('hasil/samples.csv', encoding='ISO-8859-1')

In [27]:
from nltk.corpus import stopwords

stopwords_id = stopwords.words('indonesian')
len(stopwords_id)

758

In [29]:
# print(stopwords_id)

['ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumlah', 'berkali-kali', 'berkata', 'berkehendak', 'berkeinginan'

In [28]:
# Buat fungsi untuk langkah stopwords removal

def remove_stopwords(text):
  clean_word = []
  all_text = text.split()
  for word in all_text:
    if word not in stopwords_id:
      clean_word.append(word)
  return ' '.join(clean_word)

In [33]:
samples['stopwords'] = samples['text_lower'].progress_apply(lambda x: remove_stopwords(x))

  0%|          | 0/2324 [00:00<?, ?it/s]

## Normalisasi Kata

In [31]:
# Download corpus singkatan
# !pip install wget
# import wget
# wget.download('https://raw.githubusercontent.com/ksnugroho/klasifikasi-spam-sms/master/data/key_norm.csv','dataset/normalisasi.csv')

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=9ff960d39acc32eabd596f6f50854a71dcc4832292680502b2140b25c4ca4ad4
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\8b\f1\7f\5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


'dataset/normalisasi (1).csv'

In [34]:
key_norm = pd.read_csv('dataset/normalisasi.csv')

# Buat fungsi untuk melakukan word normalization
def normalisasi(text):
  text = ' '.join([key_norm[key_norm['singkat'] == word]['hasil'].values[0] if (key_norm['singkat'] == word).any() else word for word in text.split()])
  text = str.lower(text)
  return text

In [35]:
samples['norm_content'] = samples['stopwords'].progress_apply(lambda x: normalisasi(x))

  0%|          | 0/2324 [00:00<?, ?it/s]

In [36]:
def stemming(text):
    text = stemmer.stem(text)
    return text

In [37]:
samples['stem_content'] = samples['norm_content'].progress_apply(lambda x: stemming(x))

  0%|          | 0/2324 [00:00<?, ?it/s]

In [38]:
samples.head()

,tanggal,text,likes,replies,quotes,retweets,searchQuery,timestamp,user_location,text_lower,stopwords,norm_content,stem_content
0,11/21/2023,Analisis yang keren banget dari Bang Hasan Nas...,135,32,5,84,#indonesiamaju,2023-11-21T17:31:00.000Z,"DKI Jakarta, Indonesia",analisis yang keren banget dari bang hasan nas...,analisis keren banget bang hasan nasbi nih!!! ...,analisis keren banget bang hasan nasbi nih!!! ...,analisis keren banget bang hasan nasbi nih men...
1,11/21/2023,"Mari kita bersatu, kita fokus melihat ke depan...",256,69,5,103,#indonesiamaju,2023-11-21T09:06:00.000Z,Indonesia Raya,"mari kita bersatu, kita fokus melihat ke depan...","mari bersatu, fokus fokus mencari putra putri ...","mari bersatu, fokus fokus mencari putra putri ...",mari satu fokus fokus cari putra putri baik ba...
2,11/22/2023,Gubernur BI : Pertumbuhan Ekonomi Indonesia Te...,1,0,0,0,#indonesiamaju,2023-11-22T06:40:00.000Z,"Bangka Belitung, Indonesia",gubernur bi : pertumbuhan ekonomi indonesia te...,gubernur bi : pertumbuhan ekonomi indonesia te...,gubernur bi : pertumbuhan ekonomi indonesia te...,gubernur bi tumbuh ekonomi indonesia baik indo...
3,11/22/2023,Gubernur BI : Pertumbuhan Ekonomi Indonesia Te...,1,0,0,0,#indonesiamaju,2023-11-22T06:39:00.000Z,"Bangka Belitung, Indonesia",gubernur bi : pertumbuhan ekonomi indonesia te...,gubernur bi : pertumbuhan ekonomi indonesia te...,gubernur bi : pertumbuhan ekonomi indonesia te...,gubernur bi tumbuh ekonomi indonesia baik indo...
4,11/22/2023,Stabilitas Pertumbuhan Ekonomi Indonesia \n\n#...,1,0,0,0,#indonesiamaju,2023-11-22T06:39:00.000Z,"Bangka Belitung, Indonesia",stabilitas pertumbuhan ekonomi indonesia \n\n#...,stabilitas pertumbuhan ekonomi indonesia #ekon...,stabilitas pertumbuhan ekonomi indonesia #ekon...,stabilitas tumbuh ekonomi indonesia ekonomi in...


In [39]:
samples.describe()

,likes,replies,quotes,retweets
count,2324.000000,2324.000000,2324.000000,2324.000000
mean,17.766351,2.959122,0.523236,6.558520
std,109.387089,15.704535,4.550938,35.016625
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,2.000000,0.000000,0.000000,1.000000
max,3398.000000,364.000000,130.000000,745.000000


In [40]:
print(samples)

         tanggal                                               text  likes  \
0     11/21/2023  Analisis yang keren banget dari Bang Hasan Nas...    135   
1     11/21/2023  Mari kita bersatu, kita fokus melihat ke depan...    256   
2     11/22/2023  Gubernur BI : Pertumbuhan Ekonomi Indonesia Te...      1   
3     11/22/2023  Gubernur BI : Pertumbuhan Ekonomi Indonesia Te...      1   
4     11/22/2023  Stabilitas Pertumbuhan Ekonomi Indonesia \n\n#...      1   
...          ...                                                ...    ...   
2319  11/18/2023  Wkwkw emank boleh sereceh ini \n\n #amin #Anie...    594   
2320  11/21/2023  #?????? #???? #??? #trending #viral #amin #maj...      1   
2321  11/21/2023  Makna salam demokrasi Ganjar Mahfud MD\n\n.\n\...      4   
2322  11/21/2023  Semua akan #anies pada waktunya.... InsyaAllah...      0   
2323  11/21/2023  ... maka kritik itulah sebenarnya yang membuat...     67   

      replies  quotes  retweets     searchQuery                

## FILTERING

In [41]:
def filtering(text):
  text = text.lower()                                   # Lowercase all sentences
      #   text = text.strip()                                   # Remove whitespace
  text = re.sub('[-+]?[0-9]+', ' ', text)               # Remove numbers 
  text = re.sub(r'https?://\S+|www\.\S+', ' ', text)    # Remove URLs
  text = re.sub(r"pic.twitter.com\S+", ' ', text)       # Remove custom URLs for twitter
  text = re.sub(r'\@([\w]+)',' ', text)                 # Remove Mention @
  text = re.sub(r'\#([\w]+)',' ', text)                 # Remove #TAGAR
  text = re.sub('\S*@\S*\s?', ' ', text)                # Remove email
  text = re.sub(r'[^\w\s]', ' ', text)                  # Remove punctuation
      #   text = re.sub(r'\b\w{1,3}\b','',text)                 #Remove n-chars,Remove less than 3 chars, minimum 4 character allowed "\b[a-zA-Z0-9]{3}\b"
  text = re.sub(r'[!$%^&*@#()_+|~=`{}\[\]%\-:";\'<>?,.\/]', ' ', text)  # Tahap-5: simbol
      #   text = re.sub(r'[0-9]+','', text)                     # Tahap-6: angka
  text = re.sub(r'([a-zA-Z])\1\1','\\1', text)          # Tahap-7: koreksi duplikasi tiga karakter beruntun atau lebih (contoh. yukkk)
  text = re.sub(' +',' ', text)                         #remove multiple whitespace
  text = re.sub(r'^[ ]|[ ]$','', text)                  # Tahap-9: spasi di awal dan akhir kalimat

  # text = re.sub('\b[a-zA-Z0-9]{3}\b','',text)
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore') # Remove non-ascii character
  return text

In [42]:
samples['filtering'] = samples['stem_content'].progress_apply(lambda x: filtering(x))


  0%|          | 0/2324 [00:00<?, ?it/s]

In [64]:
# samples['clean_stemming'] = samples['filtering'].progress_apply(lambda x: stemming(x))

  0%|          | 0/9 [00:00<?, ?it/s]

In [43]:
samples.to_csv( "hasil/dataset_after_resmikpu_clean_content.csv", index=False, encoding='utf-8-sig')


In [ ]:
# samples.to_csv( "hasil/samples.csv", index=False, encoding='utf-8-sig')

In [44]:
def text_preprocessing(text):
  text = text.lower()                                   # Lowercase all sentences
      #   text = text.strip()                                   # Remove whitespace
  text = re.sub('[-+]?[0-9]+', ' ', text)               # Remove numbers 
  text = re.sub(r'https?://\S+|www\.\S+', ' ', text)    # Remove URLs
  text = re.sub(r"pic.twitter.com\S+", ' ', text)       # Remove custom URLs for twitter
  text = re.sub(r'\@([\w]+)',' ', text)                 # Remove Mention @
  text = re.sub(r'\#([\w]+)',' ', text)                 # Remove #TAGAR
  text = re.sub('\S*@\S*\s?', ' ', text)                # Remove email
  text = re.sub(r'[^\w\s]', ' ', text)                  # Remove punctuation
      #   text = re.sub(r'\b\w{1,3}\b','',text)                 #Remove n-chars,Remove less than 3 chars, minimum 4 character allowed "\b[a-zA-Z0-9]{3}\b"
  text = re.sub(r'[!$%^&*@#()_+|~=`{}\[\]%\-:";\'<>?,.\/]', ' ', text)  # Tahap-5: simbol
      #   text = re.sub(r'[0-9]+','', text)                     # Tahap-6: angka
  text = re.sub(r'([a-zA-Z])\1\1','\\1', text)          # Tahap-7: koreksi duplikasi tiga karakter beruntun atau lebih (contoh. yukkk)
  text = re.sub(' +',' ', text)                         #remove multiple whitespace
  text = re.sub(r'^[ ]|[ ]$','', text)                  # Tahap-9: spasi di awal dan akhir kalimat

  # text = re.sub('\b[a-zA-Z0-9]{3}\b','',text)
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore') # Remove non-ascii character
  word_tokens = word_tokenize(text) # Word tokenize
  

  # Define Indonesian stopwords removal
  stop_words = stopwords.words('indonesian')  # NLTK Indonesian stopwords
  clean_words = [word for word in word_tokens if word not in stop_words] # stopwords removal
  clean_words = ' '.join(clean_words)

  words = set(nltk.corpus.words.words())
  word_cleaner = word_tokenize(clean_words)
  cleaner_words = [w for w in word_cleaner if w.lower() in words or not w.isalpha()] #remove non english
  cleaner = ' '.join(cleaner_words)

  #stemming with snowball update 07072022 stemmer terakhir
#   stem_token=word_tokenize(cleaner)
#   stem_words = [snowball.stem(w) for w in stem_token]
#   stem_clean= ' '.join(stem_words)

  #update13072022 pake lemmatization TextBlob
  tblob=TextBlob(clean_words)
  lemma= [Word(word).lemmatize("v") for word in tblob.words]
  lemma_clean= ' '.join(lemma)

  #   cleaner = " ".join(w for w in nltk.wordpunct_tokenize(clean_words) \
        #   if w.lower() in words or not w.isalpha())

  return lemma_clean